In [ ]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
    try:
        tf.config.set_logical_device_configuration(
            gpus[0],
            [tf.config.LogicalDeviceConfiguration(memory_limit=6*1024)])
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)


In [ ]:
# Data processing and model train

from libs.ml import AtmosphericalData
from tensorflow import keras
from keras.src.callbacks import EarlyStopping
from keras.regularizers import l2
def train_model(target_column:str, date_col: str, train_column: str, n_steps:int, training_args: dict, model_args: list, compile_args: dict = None):
    ## Read data and preprocessing

    inside = []
    for v in model_args:
        layer = getattr(keras.layers, v['layer'])

        inside.append(layer(v['units'], **v['args']))

    model = keras.models.Sequential(inside)

    ad = AtmosphericalData(model=model, path=f"./data/{target_column}/{target_column}_decomposition.csv", use_cudnn=False)
    del model

    print(ad.df.columns)

    ad.convert_date(date_col=date_col)
    ad.target_selection(train_column)
    ad.data_preprocessing(n_steps=n_steps, train_vol=0.9)

    ## Train data
    ad.train_data(
        epochs=training_args['epochs'],
        batch_size=training_args['batch_size'],
        validation_split=training_args['validation_split'],
        verbose=training_args['verbose'],
        **compile_args
    )


    ## Test trained model
    ad.test_model()


    ## Plot comparison
    ad.plot_tests()

    return ad

In [ ]:
target = 'ppt(mm)'

In [ ]:
n_steps_r = 3

model_args_r = [
    {'layer': 'Input', 'units': (n_steps_r, 3), 'args': {}},
    {'layer': 'LSTM', 'units':16, 'args': {'return_sequences': False, 'use_cudnn': False}},
    {'layer': 'Dense', 'units': 1, 'args': {}},
]

compile_args_r = {
    'optimizer': 'adam',
    'loss': 'mse',
    'metrics': ['mae'],
}

training_args_r = {
    'epochs': 150,
    'batch_size': 12,
    'validation_split': 0.2,
    'callbacks': [EarlyStopping(patience=20, restore_best_weights=True)],
    'verbose': 1
}

ad_r = train_model(
    target_column=target,
    date_col='date',
    train_column='resid',
    n_steps=n_steps_r,
    training_args=training_args_r,
    model_args=model_args_r,
    compile_args=compile_args_r
)

In [ ]:
n_steps_t = 24
model_args_t = [
    {'layer': 'InputLayer', 'units': (n_steps_t, 3), 'args': {}},  # Un solo feature (trend)
    {'layer': 'LSTM', 'units': 128, 'args': {'return_sequences': True, 'use_cudnn':False, 'dropout': 0.0}},
    {'layer': 'LSTM', 'units': 64, 'args': {'return_sequences': False, 'use_cudnn':False, 'dropout': 0.0}},
    {'layer': 'Dense', 'units': 32, 'args': {'activation': 'relu', 'kernel_regularizer': l2(1e-4)}},
    {'layer': 'Dense', 'units': 1, 'args': {}},
]

compile_args_t = {
    'optimizer': 'adam',
    'loss': 'mse',
    'metrics': ['mse'],
}

training_args_t = {
    'epochs': 300,
    'batch_size': 32,
    'validation_split': 0.15,
    'callbacks': [EarlyStopping(patience=30, restore_best_weights=True)],
    'verbose': 1,
}

ad_t = train_model(
    target_column=target,
    date_col='date',
    train_column='trend',
    n_steps=n_steps_t,
    training_args=training_args_t,
    model_args=model_args_t,
    compile_args=compile_args_t
)


In [ ]:
n_steps_s = 12

model_args_s = [
    {'layer': 'Input', 'units': (n_steps_s, 3), 'args': {}},
    {'layer': 'LSTM', 'units':32, 'args': {'return_sequences': True, 'use_cudnn': False}},
    {'layer': 'LSTM', 'units':16, 'args': {'return_sequences': False, 'use_cudnn': False}},
    {'layer': 'Dense', 'units': 1, 'args': {}},
]

compile_args_s = {
    'optimizer': 'adam',
    'loss': 'mse',
    'metrics': ['mse'],
}

training_args_s = {
    'epochs': 150,
    'batch_size': 16,
    'validation_split': 0.15,
    'callbacks': [EarlyStopping(patience=20, restore_best_weights=True)],
    'verbose': 1
}

ad_s = train_model(
    target_column=target,
    date_col='date',
    train_column='seasonal',
    n_steps=n_steps_s,
    training_args=training_args_s,
    model_args=model_args_s,
    compile_args=compile_args_s
)


In [ ]:
n_steps_o = 24

model_args_o = [
    {'layer': 'Input', 'units': (n_steps_o, 3), 'args': {}},
    {'layer': 'LSTM', 'units':32, 'args': {'return_sequences': False, 'use_cudnn': False}},
    {'layer': 'Dense', 'units': 1, 'args': {}},
]

compile_args_o = {
    'optimizer': 'adam',
    'loss': 'mse',
    'metrics': ['mse'],
}

training_args_o = {
    'epochs': 150,
    'batch_size': 24,
    'validation_split': 0.1,
    'callbacks': [EarlyStopping(patience=20, restore_best_weights=True)],
    'verbose': 0
}

ad_o = train_model(
    target_column=target,
    date_col='date',
    train_column='observed',
    n_steps=n_steps_o,
    training_args=training_args_o,
    model_args=model_args_o,
    compile_args=compile_args_o
)

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
forecast_steps = 24
series = ad_o.df.set_index('date')['observed']

# División manual
train = series[:'2016-08']  # hasta diciembre 2020
test = series['2016-09':]   # desde enero 2021

# Ajustar solo con train
model = SARIMAX(series, order=(1,1,1), seasonal_order=(1,1,1,12))
result = model.fit()

# Predecir en conjunto de prueba (in-sample)
start = test.index[0]
end = test.index[-1]
pred_test = result.predict(start=start, end=end)

# Calcular métricas
rmse = np.sqrt(mean_squared_error(test, pred_test))
mae = mean_absolute_error(test, pred_test)
print("%RMSE:", rmse * 100)
print("%MAE:", mae * 100)

# (Opcional) Predecir pasos futuros después de test
# forecast = result.get_forecast(steps=24)
# pred_future = forecast.predicted_mean
# pred_ci = forecast.conf_int()

# Graficar
plt.figure(figsize=(12,6))
plt.plot(pred_test.index, test, label='Real')
plt.plot(pred_test.index, pred_test, label='Predicción Test')
plt.legend()
plt.show()


forecast = result.get_forecast(steps=24)
pred_future = forecast.predicted_mean
pred_ci = forecast.conf_int()


plt.figure(figsize=(12,6))
plt.plot(series.index, series, label='Real')
plt.plot(pred_future.index, pred_future.values, label='Forecast', color='green')
plt.title(f"ARIMA Forecast {forecast_steps} pasos")
plt.legend()
plt.grid()
plt.show()

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Tu serie mensual
series = ad_o.df.set_index('date')['observed']

# División manual en train/test
train = series[:'2016-08']
test = series['2016-09':]

# Ajustar modelo ARIMA solo con train
# order = (p,d,q) -> ajusta según ACF/PACF o prueba heurística
model = ARIMA(series, order=(1,1,1))
result = model.fit()

# Predicción en el conjunto de prueba
pred_test = result.predict(start=test.index[0], end=test.index[-1], dynamic=False)

# Métricas
rmse = np.sqrt(mean_squared_error(test, pred_test))
mae = mean_absolute_error(test, pred_test)
print("%RMSE:", rmse * 100)
print("%MAE:", mae * 100)

# Graficar real vs predicción test
plt.figure(figsize=(12,6))
plt.plot(pred_test.index, test, label='Real')
plt.plot(pred_test.index, pred_test, label='Predicción Test', color='red')
plt.title("Predicción ARIMA Test")
plt.xlabel("Fecha")
plt.ylabel('Valor')
plt.legend()
plt.grid()
plt.show()

# Predicción de pasos futuros (out-of-sample)
forecast_steps = 24
forecast = result.get_forecast(steps=forecast_steps)
pred_future = forecast.predicted_mean
pred_ci = forecast.conf_int()

plt.figure(figsize=(12,6))
plt.plot(series.index, series, label='Real')
plt.plot(pred_future.index, pred_future.values, label='Forecast', color='green')
plt.title(f"ARIMA Forecast {forecast_steps} pasos")
plt.legend()
plt.grid()
plt.show()


In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt

# Tomar la longitud mínima entre los arrays para que coincidan
min_len = min(
    len(ad_r._true_rescaled),
    len(ad_t._true_rescaled),
    len(ad_s._true_rescaled),
    len(pred_test.values)
)

# Convertir todo a 1D
r_true = ad_r._true_rescaled[-min_len:].flatten()
t_true = ad_t._true_rescaled[-min_len:].flatten()
s_true = ad_s._true_rescaled[-min_len:].flatten()

s_pred = ad_s._pred_rescaled[-min_len:].flatten()
t_pred = ad_t._pred_rescaled[-min_len:].flatten()
r_pred = pred_test.values[-min_len:].flatten()  # SARIMA trend o residual según tu modelo

# Sumar elemento a elemento
true_rescaled = r_true + t_true + s_true
pred_rescaled = r_pred + t_pred + s_pred

# Calcular métricas
rmse = np.sqrt(mean_squared_error(true_rescaled, pred_rescaled))
mae = mean_absolute_error(true_rescaled, pred_rescaled)
print("%RMSE:", rmse * 100)
print("%MAE:", mae * 100)

# Fechas
dates = ad_t._test_dates[-min_len:]

# Graficar
plt.figure(figsize=(14,5))
plt.plot(dates, true_rescaled, label="Real", alpha=0.5)
plt.plot(dates, pred_rescaled, label="Predicción", color="red", linewidth=2)
plt.title("Predicción LSTM + SARIMA (Trend + Seasonal + Residual)")
plt.xlabel(ad_t._date_col)
plt.ylabel(ad_t._target_col)
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Encontrar la longitud mínima entre las predicciones verdaderas de los modelos
min_len = min(
    len(ad_r._true_rescaled),
    len(ad_t._true_rescaled),
    len(ad_s._true_rescaled)
)

# Recortar las verdaderas y predichas para que tengan la misma longitud (solo trend + seasonal)
true_rescaled = (
    ad_t._true_rescaled[-min_len:]
    + ad_s._true_rescaled[-min_len:]
    + ad_r._true_rescaled[-min_len:]
)

pred_rescaled = (
    ad_t._pred_rescaled[-min_len:]
    + ad_s._pred_rescaled[-min_len:]
    + ad_r._pred_rescaled[-min_len:]
)

# Aplanar los arrays para matplotlib
true_rescaled = true_rescaled.flatten()
pred_rescaled = pred_rescaled.flatten()

# Calcular métricas
rmse = np.sqrt(mean_squared_error(true_rescaled, pred_rescaled))
mae = mean_absolute_error(true_rescaled, pred_rescaled)
print("%RMSE:", rmse * 100)
print("%MAE:", mae * 100)

# Usar las fechas del modelo trend para el eje x, recortadas al mismo rango
dates = ad_t._test_dates[-min_len:]

# Graficar
plt.figure(figsize=(14,5))
plt.plot(dates, true_rescaled, label="Real", alpha=0.5)
plt.plot(dates, pred_rescaled, label="Predicción", color="red", linewidth=2)
plt.title("Predicción LSTM (Trend + Seasonal)")
plt.xlabel(ad_t._date_col)
plt.ylabel(ad_t._target_col)
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
def build_dates(year):
    return [f'{year}{month}01' if month >= 10 else f'{year}0{month}01' for month in range(1, 13)]
dates_input = build_dates(2023)
dates_input = dates_input + build_dates(2024)
predicts = []
predicts1 = []
for date in dates_input:
    _predict = ad_r.predict_for_date(date)
    _predict += ad_t.predict_for_date(date)
    _predict += ad_s.predict_for_date(date)
    predicts.append(_predict)

    _predict = ad_o.predict_for_date(date)
    predicts1.append(_predict)

forecast = result.get_forecast(steps=24)
pred_future = forecast.predicted_mean.values
pred_ci = forecast.conf_int()
plt.figure(figsize=(14,5))
plt.plot(pd.to_datetime(dates_input, format='%Y%m%d'), predicts, label="Predict S+T+R", color='white', linewidth=2)
plt.plot(pd.to_datetime(dates_input, format='%Y%m%d'), predicts1, label="Predict Obs", color='green', linewidth=1)
plt.plot(pd.to_datetime(dates_input, format='%Y%m%d'), pred_future, label="SARIMA", color='red', linewidth=3)
#plt.plot(ad_r.df['date'], ad_r.df['observed'], label="Real", color="red", linewidth=2)
plt.title("Predicción LSTM de " + target)
plt.xlabel("Fecha")
plt.ylabel(target)
plt.legend()
plt.grid()
plt.tight_layout()